In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push
instrument = "LSSTComCam"
observation_reason = "ALL"
observation_type = "acq"  # TODO: "science", "acq", default="ALL"
science_program = "ALL"

In [2]:
# IMPORT everything
import os
import datetime as dt
import pandas as pd
from IPython.display import HTML, display

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
#! import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md

from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports

In [3]:
# Set default env to "usdf" and try before PUSH to repo.

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf
)  # TODO try with "usdf" before push (else "summit")

# Normalize Parameters (both explicit Times Squares params and implicit ones)
observation_reason = None if observation_reason=='ALL' else observation_reason
observation_type = None if observation_type=='ALL' else observation_type
science_program = None if science_program=='ALL' else science_program
params = dict(
    day_obs=day_obs,
    instrument=instrument,
    observation_reason=observation_reason,
    observation_type=observation_type,
    science_program=science_program,
)

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = 1
verbose=False

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [5]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
)
allrep = AllReports(allsrc=allsrc)
src_exp = allsrc.exp_src

In [6]:
#! allrep.exp_rep.time_log_as_markdown()

In [7]:
df = src_exp.exposure_detail(instrument=instrument,
                            science_program=science_program,
                            observation_reason=observation_reason,
                            observation_type=observation_type
                           )
if observation_reason:
    md(f'# Observation Reason: {observation_reason}')
if observation_type:
    md(f'# Observation Type: {observation_type}')
if science_program:
    md(f'# Science Program: {science_program}')
md(f'The number of exposures in this filtered result is {len(df.index)}')
display(HTML(df.to_html(index=False)))

# Observation Type: acq

The number of exposures in this filtered result is 124

exposure flag,obs id,timespan begin,seq num,observation type,observation reason,science program,exposure time
unknown,CC_O_20241119_000337,2024-11-20T07:22:54.000983,337,acq,acq,BLOCK-T155,0.0
unknown,CC_O_20241119_000336,2024-11-20T07:19:30.564991,336,acq,infocus,BLOCK-T155,0.0
unknown,CC_O_20241119_000333,2024-11-20T07:16:48.884999,333,acq,acq,BLOCK-T155,0.0
unknown,CC_O_20241119_000295,2024-11-20T07:07:14.810986,295,acq,infocus,BLOCK-T298,0.0
unknown,CC_O_20241119_000292,2024-11-20T07:04:44.257990,292,acq,infocus,BLOCK-T298,0.0
unknown,CC_O_20241119_000289,2024-11-20T07:02:13.840017,289,acq,infocus,BLOCK-T298,0.0
unknown,CC_O_20241119_000286,2024-11-20T06:59:07.540002,286,acq,acq,BLOCK-T286,0.0
unknown,CC_O_20241119_000285,2024-11-20T06:57:19.744006,285,acq,infocus,BLOCK-T286,0.0
unknown,CC_O_20241119_000282,2024-11-20T06:54:43.905986,282,acq,infocus,BLOCK-T286,0.0
unknown,CC_O_20241119_000279,2024-11-20T06:52:08.310008,279,acq,infocus,BLOCK-T286,0.0


| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |

-----------------

# Developer Only Section

In [8]:
display(params)  
print(src_exp)
print(f'{instrument=}, {science_program=}, {observation_reason=}, {observation_type=}')

{'day_obs': 'YESTERDAY',
 'instrument': 'LSSTComCam',
 'observation_reason': None,
 'observation_type': 'acq',
 'science_program': None}

https://usdf-rsp-dev.slac.stanford.edu: 2024-11-19 to 2024-11-20 [2500] exposurelog endpoints=['instruments', 'exposures', 'messages']
instrument='LSSTComCam', science_program=None, observation_reason=None, observation_type='acq'


In [9]:
print({k: len(v) for k,v in src_exp.exposures.items()})

{'LSSTComCamSim': 0, 'LATISS': 0, 'LSSTComCam': 422, 'LSSTCam': 0}


In [10]:
#src_exp.exposures['LATISS']